# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [13]:
! pip install -q schedule pytest # установка библиотек, если ещё не

In [1]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule
from bs4 import BeautifulSoup


## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [12]:
 def get_book_data(book_url: str) -> dict:
    """
    Парсит данные о книге с указанной страницы сайта Books to Scrape.
    
    Функция получает HTML-страницу книги, извлекает основную информацию о книге
    (название, цена, рейтинг, наличие, описание) и дополнительные характеристики
    из таблицы Product Information.

    Args:
        book_url (str): URL-адрес страницы книги для парсинга
        
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    try:
        # Отправляем запрос к странице книги
        response = requests.get(book_url, timeout = 5)
        response.raise_for_status()  # Проверяем успешность запроса
        
        # Создаем объект BeautifulSoup для парсинга
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Словарь для хранения данных о книге
        book_data = {}
        
        # 1. Название книги
        title_element = soup.find('h1')
        book_data['title'] = title_element.text.strip() if title_element else 'N/A'
        
        # 2. Цена
        price_element = soup.find('p', class_='price_color')
        book_data['price'] = price_element.text.strip() if price_element else 'N/A'
        
        # 3. Рейтинг
        rating_element = soup.find('p', class_='star-rating')
        if rating_element:
            rating_classes = rating_element.get('class', [])
            # Ищем класс, соответствующий рейтингу (One, Two, Three, Four, Five)
            rating = next((cls for cls in rating_classes if cls in ['One', 'Two', 'Three', 'Four', 'Five']), 'N/A')
            book_data['rating'] = rating
        else:
            book_data['rating'] = 'N/A'
        
        # 4. Наличие и количество
        stock_element = soup.find('p', class_='instock availability')
        book_data['stock'] = stock_element.text.strip() if stock_element else 'N/A'
        
        # 5. Описание
        description_element = soup.find('div', id='product_description')
        if description_element:
            # Описание находится в следующем sibling элементе
            description = description_element.find_next_sibling('p')
            book_data['description'] = description.text.strip() if description else 'N/A'
        else:
            book_data['description'] = 'N/A'
        
        # 6. Дополнительные характеристики из таблицы Product Information
        product_table = soup.find('table', class_='table table-striped')
        if product_table:
            rows = product_table.find_all('tr')
            for row in rows:
                header = row.find('th')
                value = row.find('td')
                if header and value:
                    key = header.text.strip().lower().replace(' ', '_')
                    book_data[key] = value.text.strip()
        
        return book_data
        
    except requests.RequestException as e:
        print(f"Ошибка при запросе к {book_url}: {e}")
        return None
    except Exception as e:
        print(f"Ошибка при парсинге данных: {e}")
        return None
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [6]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': '£51.77',
 'rating': 'Three',
 'stock': 'In stock (22 available)',
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place t

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [16]:
def scrape_books(is_save=False) -> list:
    """
    Проходит по всем страницам каталога Books to Scrape и парсит данные о книгах.

    Использует функцию get_book_data для получения информации о каждой книге.
    При необходимости сохраняет результат в файл books_data.txt.

    Args:
        save_to_file (bool): Если True, сохраняет результаты в файл books_data.txt.
        
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    base_url = "http://books.toscrape.com/catalogue/page-{}.html"
    books_data = []
    page_number = 1

    while True:
        url = base_url.format(page_number)
        response = requests.get(url)

        # Если страница не найдена — значит, книги закончились
        if response.status_code == 404:
            break

        print(f"Обрабатываю страницу {page_number}...")

        soup = BeautifulSoup(response.content, "html.parser")
        book_links = soup.select("h3 > a")

        if not book_links:
            break

        # Проходим по всем книгам на странице
        for link in book_links:
            relative_url = link.get("href")
            book_url = "http://books.toscrape.com/catalogue/" + relative_url.replace("../", "")
            book_data = get_book_data(book_url)

            if book_data:
                books_data.append(book_data)

            time.sleep(1)

        page_number += 1

    # Сохранение в файл, если флаг установлен
    if is_save:
        try:
            with open("books_data.txt", "w", encoding="utf-8") as file:
                for book in books_data:
                    file.write(str(book) + "\n")
            print("Данные успешно сохранены в books_data.txt")
        except Exception as e:
            print(f"Ошибка при сохранении файла: {e}")

    print(f"Всего собрано книг: {len(books_data)}")
    return books_data
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [18]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

Обрабатываю страницу 1...
Обрабатываю страницу 2...
Обрабатываю страницу 3...
Обрабатываю страницу 4...
Обрабатываю страницу 5...
Обрабатываю страницу 6...
Обрабатываю страницу 7...
Обрабатываю страницу 8...
Обрабатываю страницу 9...
Обрабатываю страницу 10...
Обрабатываю страницу 11...
Обрабатываю страницу 12...
Обрабатываю страницу 13...
Обрабатываю страницу 14...
Обрабатываю страницу 15...
Обрабатываю страницу 16...
Обрабатываю страницу 17...
Обрабатываю страницу 18...
Обрабатываю страницу 19...
Обрабатываю страницу 20...
Обрабатываю страницу 21...
Обрабатываю страницу 22...
Обрабатываю страницу 23...
Обрабатываю страницу 24...
Обрабатываю страницу 25...
Обрабатываю страницу 26...
Обрабатываю страницу 27...
Обрабатываю страницу 28...
Обрабатываю страницу 29...
Обрабатываю страницу 30...
Обрабатываю страницу 31...
Обрабатываю страницу 32...
Обрабатываю страницу 33...
Обрабатываю страницу 34...
Обрабатываю страницу 35...
Обрабатываю страницу 36...
Обрабатываю страницу 37...
Обрабатыва

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
def job_with_save():
    """
    Функция-обертка, которая вызывает парсер
    с флагом is_save=True.
    """
    print("--------------------")
    print(f"[{time.ctime()}] Запускаю плановый парсинг...")
    try:
        scrape_books(is_save=True)
        print(f"[{time.ctime()}] Парсинг успешно завершен.")
    except Exception as e:
        print(f"[{time.ctime()}] Ошибка во время парсинга: {e}")
    print("--------------------")

schedule.clear()

schedule.every().day.at("19:00").do(job_with_save)

print(f"[{time.ctime()}] Планировщик запущен.")
print("Ожидаю времени для запуска задачи...")

try:
    while True:
        schedule.run_pending()
        now = time.localtime()
        if now.tm_hour == 18 and now.tm_min >= 55:
            time.sleep(10)
        elif now.tm_hour == 18 and now.tm_min < 55:
            time.sleep(300)
        else:
            time.sleep(3600)
except KeyboardInterrupt:
    print(f"\n[{time.ctime()}] Планировщик остановлен вручную.")
            
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [27]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest tests/test_scraper.py

============================= test session starts =============================
platform win32 -- Python 3.12.7, pytest-7.4.4, pluggy-1.0.0 -- C:\Users\Artem\anaconda3\python.exe
cachedir: .pytest_cache
rootdir: D:\\\u041c\u0424\u0422\u0418 \u043c\u0430\u0433\u0438\u0441\u0442\u0440\u0430\u0442\u0443\u0440\u0430\\\u0414\u0438\u0441\u0446\u0438\u043f\u043b\u0438\u043d\u044b\\\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Python
configfile: pytest.ini
plugins: anyio-4.2.0
collecting ... collected 3 items

tests/test_scraper.py::test_scrape_books_mocked PASSED                   [ 33%]
tests/test_scraper.py::test_get_book_data_success PASSED                 [ 66%]
tests/test_scraper.py::test_get_book_data_network_failure PASSED         [100%]

============================== 3 passed in 2.26s ==============================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```